In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

from custom_datasets.rollout_push_any import RolloutPushAnyDataset
from lerobot.rollout_datasets.episode_stores import EpisodeVideoStore

from pathlib import Path
import zarr
import numpy as np
import torch
import torch.nn.functional as F
import imageio
from IPython.display import Video

In [2]:
data_path = "/home/sm/Datasets/lerobot/pusht"
output_directory = Path(data_path)
output_directory.mkdir(parents=True, exist_ok=True)

video_directory = output_directory / 'videos'
video_directory.mkdir(parents=True, exist_ok=True)

new_video_directory = output_directory / 'new_videos'
new_video_directory.mkdir(parents=True, exist_ok=True)

In [68]:
mode = 'r'
root_group = zarr.open(data_path, mode=mode)
data_group = root_group['data']

KeyError: 'data'

In [11]:
zarr_dict = {k: v for k, v in data_group.items()}

In [13]:
frame_infos = zarr_dict['observation.image'][:]

In [16]:
frame_infos

array([{'path': 'videos/observation.image_episode_YSnYvt5M.mp4', 'timestamp': 0.0},
       {'path': 'videos/observation.image_episode_YSnYvt5M.mp4', 'timestamp': 0.1},
       {'path': 'videos/observation.image_episode_YSnYvt5M.mp4', 'timestamp': 0.2},
       ...,
       {'path': 'videos/observation.image_episode_fXvLRqMz.mp4', 'timestamp': 16.1},
       {'path': 'videos/observation.image_episode_fXvLRqMz.mp4', 'timestamp': 16.2},
       {'path': 'videos/observation.image_episode_fXvLRqMz.mp4', 'timestamp': 16.3}],
      dtype=object)

In [3]:
store = EpisodeVideoStore.create_from_path(output_directory)

Connected to the exising zarr


In [4]:
dataset = store.convert_to_lerobot_dataset()

In [56]:
def read_frames(dataset, episode_index):
    from_idx = dataset.episode_data_index["from"][episode_index].item()
    to_idx = dataset.episode_data_index["to"][episode_index].item()
    
    print(f"episode {episode_index} start from index {from_idx} to index {to_idx}")
    
    frames = [dataset[idx]["observation.image"] for idx in range(from_idx, to_idx)]
    
    return torch.stack(frames, dim=0)

def torch_frames_to_video(frames: torch.Tensor):
    # Video frames are now float32 in range [0,1] channel first (c,h,w) to follow pytorch convention. To visualize
    # them, we convert to uint8 in range [0,255]
    return (frames * 255).type(torch.uint8).numpy()

def resize_image_tensor(obs):
    assert len(obs.size()) == 4  # T, C, H, W
    obs = F.interpolate(obs, size=[224, 224], mode='bilinear')
    return obs

In [44]:
episode_index = 3

frames = read_frames(dataset, episode_index)
resized_frames = resize_image_tensor(frames)


episode 3 start from index 893 to index 1188


In [57]:
numpy_frames = torch_frames_to_video(resized_frames)

In [64]:
video_group = root_group.create_dataset('video')

TypeError: create() missing 1 required positional argument: 'shape'